In [13]:
import pandas as pd

import numpy as np
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from rich.traceback import install
install()

import os
from joblib import dump
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
import pandas_profiling

In [14]:
!pwd

/Users/youheekil/Documents/github/ucdacity_c3/udacity_c3_cencus/notebook


In [18]:
path = "../data/raw/census.csv"
raw_data = pd.read_csv(path)

In [5]:
# remove space in column 
_columns = raw_data.columns
replaced_columns = []

for column in _columns:
    col = column.replace(" ", "")
    replaced_columns.append(col)

In [6]:
clean_data = raw_data.dropna()
clean_data.columns = replaced_columns

In [7]:
clean_data.columns

Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [8]:
clean_data.to_csv ("../data/clean_data.csv", index = False, header=True)


In [9]:
mapping = {
   " <=50K": 0, 
   " >50K" : 1
}

In [10]:
clean_data.loc[:, "target"] = clean_data.salary.map(mapping)

In [11]:
clean_data.target.value_counts()

0    24720
1     7841
Name: target, dtype: int64

We see that the target is skewed and thus the best metric for this binary classification problem would be Area Under the ROC Curve (AUC).

In [ ]:
def process_data(
    X, categorical_features=[], label=None, training=True, encoder=None,
    lb=None
):
    """ Process the data used in the machine learning pipeline.
    Processes the data using one hot encoding for the categorical features
    and a
    label binarizer for the labels. This can be used in either training or
    inference/validation.
    Note: depending on the type of model used, you may want to add in
    functionality that
    scales the continuous data.
    Inputs
    ------
    X : pd.DataFrame
        Dataframe containing the features and label. Columns in
        `categorical_features`
    categorical_features: list[str]
        List containing the names of the categorical features (default=[])
    label : str
        Name of the label column in `X`. If None, then an empty array will
        be returned
        for y (default=None)
    training : bool
        Indicator if training mode or inference/validation mode.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained sklearn OneHotEncoder, only used if training=False.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained sklearn LabelBinarizer, only used if training=False.
    Returns
    -------
    X : np.array
        Processed data.
    y : np.array
        Processed labels if labeled=True, otherwise empty np.array.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained OneHotEncoder if training is True, otherwise returns the
        encoder passed
        in.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained LabelBinarizer if training is True, otherwise returns the
        binarizer
        passed in.
    """

    if label is not None:
        y = X[label]
        X = X.drop([label], axis=1)
    else:
        y = np.array([])

    X_categorical = X[categorical_features].values
    X_continuous = X.drop(*[categorical_features], axis=1)

    if training is True:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        lb = LabelBinarizer()
        X_categorical = encoder.fit_transform(X_categorical)
        y = lb.fit_transform(y.values).ravel()
    else:
        X_categorical = encoder.transform(X_categorical)
        try:
            y = lb.transform(y.values).ravel()
        # Catch the case where y is None because we're doing inference.
        except AttributeError:
            pass

    X = np.concatenate([X_continuous, X_categorical], axis=1)
    return X, y, encoder, lb

In [20]:
def get_cat_features():
    cat_features = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]
    return cat_features

In [13]:
train, test = train_test_split(clean_data, test_size=0.20)


X_train, y_train, encoder, lb = process_data(
        train, categorical_features=get_cat_features(), label="salary", training=True
)


In [14]:
X_test, y_test, encoder, lb = process_data(test, categorical_features=get_cat_features(), label="salary", training=False)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_40758/3538399110.py:1 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_40758/3538399110.py'          │
│ /var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_40758/3479765932.py:56 in      │
│ process_data                                                                              │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_40758/3479765932.py'          │
│                                                                                           │
│ /opt/anaconda3/envs/cencus/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py │
│ :506 in transform                                                                         │
│                                                                                           │
│    503 │   │   │   Transformed input. If `sparse=True`, a sparse matrix will be           │
│    504 │   │   │   returned.                                                              │
│    505 │   │   """                                                                        │
│ ❱  506 │   │   check_is_fitted(self)                                                      │
│    507 │   │   # validation of X happens in _check_X called by _transform                 │
│    508 │   │   warn_on_unknown = self.handle_unknown == "ignore" and self.drop is not Non │
│    509 │   │   X_int, X_mask = self._transform(                                           │
│                                                                                           │
│ /opt/anaconda3/envs/cencus/lib/python3.8/site-packages/sklearn/utils/validation.py:1222   │
│ in check_is_fitted                                                                        │
│                                                                                           │
│   1219 │   │   ]                                                                          │
│   1220 │                                                                                  │
│   1221 │   if not fitted:                                                                 │
│ ❱ 1222 │   │   raise NotFittedError(msg % {"name": type(estimator).__name__})             │
│   1223                                                                                    │
│   1224                                                                                    │
│   1225 def check_non_negative(X, whom):                                                   │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate 
arguments before using this estimator.

In [19]:


# Optional: implement hyperparameter tuning.
def train_model(X_train, y_train):
    """
    Trains a machine learning model and returns it.

    Inputs
    ------
    X_train : np.array
        Training data.
    y_train : np.array
        Labels.
    Returns
    -------
    model
        Trained machine learning model.
    """
    # define the model
    model = RandomForestClassifier()
    # evaluate the model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    return y_pred
    

In [20]:
y_pred = train_model(X_train=X_train, y_train=y_train)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_60714/3476531834.py:1 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/fl/4yl38nys37gcpbrgk1v5qq000000gn/T/ipykernel_60714/3476531834.py'          │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'X_train' is not defined

In [21]:

def get_cat_features():
    cat_features = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]
    return cat_features



def process_data(
    X, categorical_features=[], label=None, training=True, encoder=None, lb=None):
    """ Process the data used in the machine learning pipeline.

    Processes the data using one hot encoding for the categorical features and a
    label binarizer for the labels. This can be used in either training or
    inference/validation.

    Note: depending on the type of model used, you may want to add in functionality that
    scales the continuous data.

    Inputs
    ------
    X : pd.DataFrame
        Dataframe containing the features and label. Columns in `categorical_features`
    categorical_features: list[str]
        List containing the names of the categorical features (default=[])
    label : str
        Name of the label column in `X`. If None, then an empty array will be returned
        for y (default=None)
    training : bool
        Indicator if training mode or inference/validation mode.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained sklearn OneHotEncoder, only used if training=False.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained sklearn LabelBinarizer, only used if training=False.

    Returns
    -------
    X : np.array
        Processed data.
    y : np.array
        Processed labels if labeled=True, otherwise empty np.array.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained OneHotEncoder if training is True, otherwise returns the encoder passed
        in.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained LabelBinarizer if training is True, otherwise returns the binarizer
        passed in.
    """

    if label is not None:
        y = X[label]
        X = X.drop([label], axis=1)
    else:
        y = np.array([])

    X_categorical = X[categorical_features].values
    X_continuous = X.drop(*[categorical_features], axis=1)

    if training is True:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        lb = LabelBinarizer()
        X_categorical = encoder.fit_transform(X_categorical)
        y = lb.fit_transform(y.values).ravel()
    else:
        X_categorical = encoder.transform(X_categorical)
        try:
            y = lb.transform(y.values).ravel()
        # Catch the case where y is None because we're doing inference.
        except AttributeError:
            pass

    X = np.concatenate([X_continuous, X_categorical], axis=1)
    return X, y, encoder, lb





In [23]:
from joblib import dump, load

import os
from joblib import dump
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, precision_score, recall_score
from sklearn.model_selection import train_test_split



In [38]:

def train_model(X_train, y_train):
    """
    Trains a machine learning model and returns it.

    Inputs
    ------
    X_train : np.array
        Training data.
    y_train : np.array
        Labels.
    Returns
    -------
    model
        Trained machine learning model.
    """
    # initialize the model 
    rfc = RandomForestClassifier(random_state=42)

    PARAM_GRID = { 
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy']
    }


    grid_search = GridSearchCV(
        estimator=rfc,
        param_grid=PARAM_GRID,
        scoring = 'roc_auc',
        n_jobs = 1,
        cv = 5,
        verbose=True
    )
    # fit model on training data
    grid_search.fit(X_train, y_train)

    # best model trained by GridSearch.
    best_model = grid_search.best_estimator_
    
    return best_model 

In [39]:
df = pd.read_csv("../data/processed/processed_census.csv")


In [40]:
train, test = train_test_split(df, test_size = 0.2)


In [48]:
X_train, y_train, encoder, lb = process_data(
        train, categorical_features=get_cat_features(), label="salary", training=True
    )

dump(encoder, "../model/encoder.joblib")
dump(lb, "../model/lb.joblib")

['../model/lb.joblib']

In [44]:
best_model = train_model(X_train=X_train, y_train=y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [47]:
dump(best_model, "../model/random_forest.pkl")

['../model/random_forest.pkl']

In [49]:
best_model

RandomForestClassifier(max_depth=8, n_estimators=500, random_state=42)

In [50]:
rfc1=RandomForestClassifier(
    random_state=42, 
    max_features='auto', 
    n_estimators= 500, 
    max_depth=8, 
    criterion='gini')

In [51]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=500, random_state=42)

In [52]:
pred=best_model.predict(X_train)

In [53]:
pred1 = rfc1.predict(X_train)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, pred)

0.8551904176904177

In [61]:
best_model = load("../model/random_forest.pkl")

In [68]:
cat ="3x"
cls = "dv"

precision = 10
recall = 2
fbeta = 2


with open("../notebook/data_slicing_score2.txt", "a") as f:
            f.write(f"{cat}[{cls}]\n")
            f.write(f"precision: {precision}\n")
            f.write(f"recall: {recall}\n")
            f.write(f"fbeta: {fbeta}\n")
            f.write("===================================\n")

    